# 1 Importación de librerías

In [2]:
import pandas as pd
import glob # Para unir los CSV
import os
import seaborn as sns
import matplotlib.pyplot as plt

# 2 Carga de datos y unión

In [4]:
path = r'C:\Users\Antonio\Desktop\Python\data' 
all_files = glob.glob(os.path.join(path, "PRSA_Data_*.csv"))

li = []
for filename in all_files:
    df_temp = pd.read_csv(filename)
    li.append(df_temp)

df = pd.concat(li, axis=0, ignore_index=True)

# 3 EDA

In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420768 entries, 0 to 420767
Data columns (total 18 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   No       420768 non-null  int64  
 1   year     420768 non-null  int64  
 2   month    420768 non-null  int64  
 3   day      420768 non-null  int64  
 4   hour     420768 non-null  int64  
 5   PM2.5    412029 non-null  float64
 6   PM10     414319 non-null  float64
 7   SO2      411747 non-null  float64
 8   NO2      408652 non-null  float64
 9   CO       400067 non-null  float64
 10  O3       407491 non-null  float64
 11  TEMP     420370 non-null  float64
 12  PRES     420375 non-null  float64
 13  DEWP     420365 non-null  float64
 14  RAIN     420378 non-null  float64
 15  wd       418946 non-null  object 
 16  WSPM     420450 non-null  float64
 17  station  420768 non-null  object 
dtypes: float64(11), int64(5), object(2)
memory usage: 57.8+ MB
None


In [7]:
print(df.isnull().sum())

No             0
year           0
month          0
day            0
hour           0
PM2.5       8739
PM10        6449
SO2         9021
NO2        12116
CO         20701
O3         13277
TEMP         398
PRES         393
DEWP         403
RAIN         390
wd          1822
WSPM         318
station        0
dtype: int64


# 4 Transformación y limpieza de datos

Tratamos fechas unificando en 'date' y borramos 'No'

In [10]:
df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])

df.set_index('date', inplace=True) # date como índice para posterior análisis temporal

df.drop(columns=['No', 'year', 'month', 'day', 'hour'], inplace=True)

df.head()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
date,,,,,,,,,,,,,
2013-03-01 00:00:00,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
2013-03-01 01:00:00,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2013-03-01 02:00:00,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
2013-03-01 03:00:00,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
2013-03-01 04:00:00,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin


Utilizamos interpolate para no perder información y rellenar los datos faltantes

In [12]:
cols_a_limpiar = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']

df[cols_a_limpiar] = df[cols_a_limpiar].interpolate(method='linear') #rellena los NaN calculando el valor intermedio entre n-1 y n+1

# Rellena el hueco con el último valor de dirección conocido
df['wd'] = df['wd'].ffill()

print(df.isnull().sum())

PM2.5      0
PM10       0
SO2        0
NO2        0
CO         0
O3         0
TEMP       0
PRES       0
DEWP       0
RAIN       0
wd         0
WSPM       0
station    0
dtype: int64


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 420768 entries, 2013-03-01 00:00:00 to 2017-02-28 23:00:00
Data columns (total 13 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   PM2.5    420768 non-null  float64
 1   PM10     420768 non-null  float64
 2   SO2      420768 non-null  float64
 3   NO2      420768 non-null  float64
 4   CO       420768 non-null  float64
 5   O3       420768 non-null  float64
 6   TEMP     420768 non-null  float64
 7   PRES     420768 non-null  float64
 8   DEWP     420768 non-null  float64
 9   RAIN     420768 non-null  float64
 10  wd       420768 non-null  object 
 11  WSPM     420768 non-null  float64
 12  station  420768 non-null  object 
dtypes: float64(11), object(2)
memory usage: 44.9+ MB
